In [201]:
import statsmodels.api as sm
import pandas as pd
from patsy import dmatrices
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison, tukeyhsd
import numpy as np
from statsmodels.stats.libqsturng import psturng

In [202]:
from statsmodels.formula.api import ols

# Example of 2-way ANOVA
This is made to see if class balance and batch type influences the PPV

In [203]:
moore = sm.datasets.get_rdataset("Moore", "car",cache=True) # load data

In [204]:
df = moore.data
df = df.drop(['fscore'],axis=1).rename(columns={'partner.status':'class_balance','conformity':'PPV','fcategory':'batch'})
df['batch'].replace(["low",'medium','high'],['1','2','3'],inplace=True)
df['class_balance'].replace(["low",'high'],['50-50','90-10'],inplace=True)
df.head(25)

,class_balance,PPV,batch
0,50-50,8,1
1,50-50,4,3
2,50-50,8,3
3,50-50,7,1
4,50-50,10,1
5,50-50,6,1
6,50-50,12,2
7,50-50,4,2
8,50-50,13,1
9,50-50,12,1


In our case class_balance can be "50-50" or "90-10", and batch can be "relevant first","relevant last","random"

If you don't know how to creat this table, follow this (I will not compute this here but the shape is the same):

In [205]:
df_fit = ols('PPV ~ class_balance + batch +class_balance*batch',data=df).fit()

In [206]:
anova_results = sm.stats.anova_lm(df_fit, typ=2)

In [207]:
anova_results

,sum_sq,df,F,PR(>F)
class_balance,212.213778,1.0,10.120692,0.002874
batch,11.614700,2.0,0.276958,0.759564
class_balance:batch,175.488928,2.0,4.184623,0.022572
Residual,817.763961,39.0,NaN,NaN


if PR(>F) is less than 0.05 you can consider there is an effect on PPV. After that you need to do a post-hoc analysis (for the classes that passes the test, in this example batch > 0.05 so we don't have to do the post-hoc analysis foe the batches.

# Post-hoc analysis
This is made to see which specific batch (or class balance) is significantly different than another.

In [208]:
def compute_tukey(df,target,classes):
    n_classes = len(df[classes].unique())
    degrees_freedom = len(df) - n_classes
    results_tukey = pairwise_tukeyhsd(df[target], df[classes])
    v = np.abs(results_tukey.meandiffs/ results_tukey.std_pairs)
    confidence = psturng(v, n_classes, degrees_freedom)
    #print(results_tukey.summary(),confidence)
    return results_tukey.summary(),confidence

In [209]:
(result,confidence) = compute_tukey(df,'PPV','class_balance')

In [210]:
result

group1,group2,meandiff,lower,upper,reject
50-50,90-10,4.2628,1.3555,7.1702,True


In [211]:
confidence

array([ 0.00502871])

In [212]:
(result2,confidence2) = compute_tukey(df,'PPV','batch')

In [213]:
result2

group1,group2,meandiff,lower,upper,reject
1,2,0.6667,-4.0858,5.4192,False
1,3,0.5333,-4.2192,5.2858,False
2,3,-0.1333,-4.8858,4.6192,False
